# ClusterEnsemble demonstration
Some example usage of how to build up a dataframe of galaxy cluster properties, including NFW halo profiles. Each cluster is treated as an individual, meaning we track its individual mass and redshift, and other properties. This is useful for fitting a stacked weak lensing profile, for example, where you want to avoid fitting a single average cluster mass.

In [1]:
from __future__ import absolute_import, division, print_function

%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
from astropy import units
from matplotlib import pyplot as plt

#import seaborn as sns; sns.set() 

In [2]:
from clusters import ClusterEnsemble

### Create a ClusterEnsemble object by passing in a numpy array (or list) of redshifts

In [3]:
z = np.array([0.1,0.2,0.3])
c = ClusterEnsemble(z)
c.describe

'Ensemble of galaxy clusters and their properties.'

### Display what we have so far
Below the DataFrame (which so far only contains the cluster redshifts), we see the default assumptions for the power-law slope and normalization that will be used to convert richness $N_{200}$ to mass $M_{200}$. We'll see how to change those parameters below.

In [4]:
c.show()


Cluster Ensemble:


,z
0,0.1
1,0.2
2,0.3



Mass-Richness Power Law: M200 = norm * (N200 / 20) ^ slope
   norm: 2.7e+13 solMass
   slope: 1.4


### Add richness values to the dataframe
This step will also generate $M_{200}$, $r_{200}$, $c_{200}$, scale radius $r_s$, and other parameters, assuming the scaling relation given below.

In [5]:
n200 = np.ones(3)*20.
c.n200 = n200
c.show()


Cluster Ensemble:


,z,n200,m200,r200,c200,delta_c,rs
0,0.1,20,2.700000e+13,0.612222,5.821359,12329.828385,0.105168
1,0.2,20,2.700000e+13,0.591082,5.627032,11398.735247,0.105043
2,0.3,20,2.700000e+13,0.569474,5.426060,10482.820471,0.104952



Mass-Richness Power Law: M200 = norm * (N200 / 20) ^ slope
   norm: 2.7e+13 solMass
   slope: 1.4


### Access any column of the dataframe as an array
Notice that [astropy units](http://docs.astropy.org/en/stable/units/) are present for the appropriate columns.

In [ ]:
print('z: \t', c.z)
print('n200: \t', c.n200)
print('r200: \t', c.r200)
print('m200: \t', c.m200)
print('c200: \t', c.c200)
print('rs: \t', c.rs)

### If you don't want units, you can get just the values

In [ ]:
c.r200.value

### Or access the Pandas DataFrame directly

In [ ]:
c.dataframe

### Change the redshifts or richness values
These changes will propogate to all redshift-dependant or richness-dependant cluster attributes, as appropriate.

In [ ]:
c.z = np.array([0.4,0.5,0.6])
c.dataframe

In [ ]:
c.n200 = [20,30,40]
c.show()

### Change the parameters in the mass-richness relation
Either or both of the keyword parameters "slope" and "norm" can be passed to the update_massrichrelation() method.

In [ ]:
c.update_massrichrelation(slope = 1.5)
c.show()

### Show basic table
Perhaps we don't want the fancy pandas formatting on our table, or maybe we're not working in the Jupyter notebook.

In [ ]:
c.show(notebook = False)

## Calculate $\Sigma(r)$ and $\Delta\Sigma(r)$ for NFW model
First select the radial bins in units of Mpc.

In [ ]:
rmin, rmax = 0.1, 5. #Mpc
nbins = 50
rbins = np.logspace(np.log10(rmin), np.log10(rmax), num = nbins)
#rbins

In [ ]:
c.calc_nfw(rbins)

sigma = c.sigma_nfw
deltasigma = c.deltasigma_nfw

In [ ]:
sigma

In [ ]:
fig = plt.figure(figsize=(12,5))
fig.suptitle('Centered NFW Cluster Profiles', size=30)

first = fig.add_subplot(1,2,1)
second = fig.add_subplot(1,2,2)

for rich, profile in zip(c.n200,deltasigma):
    first.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
first.set_xscale('log')
first.set_xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
first.set_ylabel('$\Delta\Sigma(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', 
                 fontsize=20)
first.set_xlim(rbins.min(), rbins.max())
first.legend(fontsize=20)


for rich, profile in zip(c.n200,sigma):
    second.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
second.set_xscale('log')
second.set_xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
second.set_ylabel('$\Sigma(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', 
                 fontsize=20)
second.set_xlim(rbins.min(), rbins.max())
second.legend(fontsize=20)


fig.tight_layout()
plt.subplots_adjust(top=0.88)

# Calculate Miscentered NFW Profiles

When the true underlying dark matter distribution is offset from the assumed cluster "center" (such as a BCG or some other center proxy), the weak lensing profiles measured around the assumed centers will be different than for the perfectly centered case. One way to account for this is to describe the cluster centroid offsets as a Gaussian distribution around the true centers. We say the probability of an offset is given by

$P(R_\mathrm{off}) = \frac{R_\mathrm{off}}{\sigma_\mathrm{off}^2}e^{-\frac{1}{2}\left(\frac{R_\mathrm{off}}{\sigma_\mathrm{off}}\right)^2}$,

which is parameterized by the width of the 2D offset distribution $\sigma_\mathrm{off} = \sqrt{\sigma_x^2 + \sigma_y^2}$. Then the measured surface mass density is given by 

$\Sigma^\mathrm{sm}(R) = \int_0^\infty \Sigma(R | R_\mathrm{off})\  P(R_\mathrm{off})\ \mathrm{d}R_\mathrm{off}$,

where

$\Sigma(R | R_\mathrm{off}) = \frac{1}{2\pi} \int_0^{2\pi} \Sigma(r')\ \mathrm{d}\theta$, 

and

$r' = \sqrt{R^2 + R_\mathrm{off}^2 - R R_\mathrm{off} \cos{\theta}}$.

More details on the cluster miscentering problem can be found in [Ford et al 2015](http://arxiv.org/abs/1409.3571), [George et al 2012](http://arxiv.org/abs/1205.4262), and [Johnston et al 2007](http://arxiv.org/abs/0709.1159).

To calculate the miscentered profiles, simply create an array of offsets in units of Mpc, and pass it to the calc_nfw method:

In [ ]:
offsets = np.array([0.1,0.1,0.1]) #same length as number of clusters

c.calc_nfw(rbins, offsets=offsets)

deltasigma_off = c.deltasigma_nfw
sigma_off = c.sigma_nfw

In [ ]:
fig = plt.figure(figsize=(12,5))
fig.suptitle('Miscentered NFW Cluster Profiles', size=30)

first = fig.add_subplot(1,2,1)
second = fig.add_subplot(1,2,2)

for rich, profile in zip(c.n200,deltasigma_off):
    first.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
first.set_xscale('log')
first.set_xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
first.set_ylabel('$\Delta\Sigma^\mathrm{sm}(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', fontsize=20)
first.set_xlim(rbins.min(), rbins.max())
first.legend(fontsize=20)


for rich, profile in zip(c.n200,sigma_off):
    second.plot(rbins, profile, label='$N_{200}=$ '+str(rich))
second.set_xscale('log')
second.set_xlabel('$r\ [\mathrm{Mpc}]$', fontsize=20)
second.set_ylabel('$\Sigma^\mathrm{sm}(r)\ [\mathrm{M}_\mathrm{sun}/\mathrm{pc}^2]$', 
                 fontsize=20)
second.set_xlim(rbins.min(), rbins.max())
second.legend(fontsize=20)


fig.tight_layout()
plt.subplots_adjust(top=0.88)

## NEW: a Python implentation of the NFW calculations 
Set the keyword parameter "use_c = False" to use Python only. Note that this method is significantly faster than the C code, for the perfectly centered case (because the C version suboptimally writes/reads to disc), but far slower for the miscentered case (which includes calculating a double integral). *Currently, the Python option calculates centered $\Sigma(r)$ and $\Delta\Sigma(r)$ profiles, and the miscentered profile $\Sigma^\mathrm{sm}(r)$, but not yet $\Delta\Sigma^\mathrm{sm}(r)$...*

In [ ]:
c.calc_nfw(rbins, use_c = False)

sigma_py = c.sigma_nfw
dsigma_py = c.deltasigma_nfw

In [ ]:
sigma_py = c.sigma_nfw
dsigma_py = c.deltasigma_nfw

In [ ]:
#check the results match
np.testing.assert_allclose(sigma_py, sigma, rtol = 10**-4)
np.testing.assert_allclose(dsigma_py, deltasigma, rtol = 10**-4)

### Python Calculation of $\Sigma^\mathrm{sm}(r)$
Even with dblquad and large epsabs, epsrel allowances, this is way slow.

In [ ]:
c.calc_nfw(rbins, offsets = offsets, use_c = False, 
           epsabs = 0.5, epsrel = 0.5)

sigma_off_py = c.sigma_nfw
#deltasigma_off_py = c.deltasigma_nfw #not yet implemented

In [ ]:
sigma_off_py

In [ ]:
#check the results match... 
np.testing.assert_allclose(sigma_off_py, sigma_off, rtol = 10**-1)

#looks like 10**-1 to 10**-2 is the level at which the offset 
#profiles match, depending on what I set the epsabs, epsrel to be.

### Timing Comparisons: Python vs C
- For centered halos, Python implementation is about 30% faster than C.
- For offset halos, need $\Delta\Sigma^\mathrm{sm}(r)$ implemented in Python for full comparison... but just the $\Sigma^\mathrm{sm}(r)$ bit is nearly two orders of magnitude faster in C than Python.

In [ ]:
time_c = %timeit -o c.calc_nfw(rbins)
time_py = %timeit -o c.calc_nfw(rbins, use_c = False)
time_offset_c = %timeit -o c.calc_nfw(rbins, offsets=offsets)
time_offset_py_justsigma = %timeit -o c.calc_nfw(rbins, \
    offsets = offsets, use_c = False, epsabs = 0.5, epsrel = 0.5)

In [ ]:
#Centered: Python vs C 
time_py.best / time_c.best

In [ ]:
#Miscentered: Python vs C 
time_offset_py_justsigma.best / time_offset_c.best

In [ ]:
#clocktime_c = %timeit -o -c c.calc_nfw(rbins)
#clocktime_py = %timeit -o -c c.calc_nfw(rbins, use_c = False)
#clocktime_offset_c = %timeit -o -c c.calc_nfw(rbins, offsets=offsets)
#clocktime_offset_py_justsigma = %timeit -o -c c.calc_nfw(rbins, \
#    offsets = offsets, use_c = False, epsabs = 0.5, epsrel = 0.5)

In [ ]:
#Centered: Python vs C (wall clock time accounts for read/write to disc)
#clocktime_py.best / clocktime_c.best

In [ ]:
#Miscentered: Python vs C 
#clocktime_offset_py_justsigma.best / clocktime_offset_c.best

### To Do: 
- speed up Python miscentered code?
- implement $\Delta\Sigma^\mathrm{sm}(r)$ in Python
- replace smd_nfw.c with cython version
- option to pass in a $M_{prelim}$ and $M_{200} = a \times$ $M_{prelim}$ relation?